### Libraries

In [1]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

import numpy as np
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [9]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Handle class imbalance

In [11]:
smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection - Model registry")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/06/08 20:00:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/818207888216497061/runs/3bef623ef3dc414b87c42181d07844fc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818207888216497061


2025/06/08 20:00:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/818207888216497061/runs/2aa807b7f3f045898cfdd3628c468869
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818207888216497061


2025/06/08 20:00:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/818207888216497061/runs/f2c5e99cc300484885fbc85134fdd2a5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818207888216497061


2025/06/08 20:00:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/818207888216497061/runs/41755fdb324e4296ba434d3478f83de0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818207888216497061


### Register the Model

In [4]:
# ! pip install ipywidgets

In [ ]:
import mlflow
import ipywidgets as widgets
from IPython.display import display, clear_output

# Step 1: Create a text widget to input run_id
run_id_widget = widgets.Text(
    value='',
    placeholder='Enter MLflow Run ID',
    description='Run ID:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

# Step 2: Output widget for displaying messages
output = widgets.Output()

# Step 3: Define the callback function when user hits Enter
def on_submit(change):
    with output:
        clear_output()  # Clear previous outputs
        run_id = run_id_widget.value.strip()
        
        if not run_id:
            print("⚠️ Please enter a valid Run ID.")
            return
        
        model_name = 'XGB-Smote'
        artifact_path = 'model'  # default unless you used another path
        model_uri = f'runs:/{run_id}/{artifact_path}'
        
        try:
            with mlflow.start_run(run_id=run_id):
                result = mlflow.register_model(model_uri=model_uri, name=model_name)
                print(f"✅ Model registered successfully!")
                print(f"Model Name: {model_name}")
                print(f"Model URI: {model_uri}")
                print(f"Version Info: {result.version}")
        except Exception as e:
            print(f"❌ Error registering model: {e}")

# Step 4: Attach the callback
run_id_widget.on_submit(on_submit)

# Step 5: Display widgets
display(run_id_widget, output)

Text(value='', description='Run ID:', layout=Layout(width='50%'), placeholder='Enter MLflow Run ID', style=Tex…

Output()

### Load the Model

In [16]:
model_version = 2
model_name = 'XGB-Smote'
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [17]:
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [18]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod" # can give any name you want

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '2' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1749395225059, current_stage='None', description=None, last_updated_timestamp=1749395225059, name='anomaly-detection-prod', run_id='41755fdb324e4296ba434d3478f83de0', run_link=None, source='models:/XGB-Smote/2', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [19]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])